In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt
import pandas as pd
import numpy as np

from scipy.signal import butter, filtfilt, stft
from scipy.fft import fft, fftfreq
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.anova import AnovaRM

from analysing import AnalysisContainer, AnalysingResults


fs = 1500
trainings = {
    0: 0,
    1: 'LE_HYP_5x12',
    2: 'LE_HYP_8x12',
    3: 'LE_MAX_5x4',
    4: 'LP_HYP_5x12'   
}

folder_path = 'raw_emg_csv'

In [2]:
df = pd.read_csv('./raw_emg_csv/all_emg.csv')

df['training_id'] = df['training_id'].astype(int)
df['training_id'] = df['training_id'].map(trainings)
df.head(2)

C:\Users\Simon\AppData\Local\Temp\ipykernel_34992\3356098989.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./raw_emg_csv/all_emg.csv')


,Unnamed: 0.1,Unnamed: 0,time,vl,rf,vm,Filtered_rf,Filtered_vl,Filtered_vm,rf_abs,vl_abs,vm_abs,Rolling_RMS_rf,Rolling_RMS_vl,Rolling_RMS_vm,training_id,rep,set,pp,rep_busy
0,0,0,1970-01-01 00:00:00.000000000,5.36975,-0.158505,-0.438854,0.241542,-0.525227,0.171729,0.241542,0.525227,0.171729,1.739555,2.964063,1.479010,LE_HYP_5x12,0.0,0,1,NaN
1,1,1,1970-01-01 00:00:00.000700000,1.70965,-0.792526,0.172208,-0.020939,-3.883819,1.034977,0.020939,3.883819,1.034977,1.731612,2.987389,1.474593,LE_HYP_5x12,0.0,0,1,NaN


In [3]:
timewindow=0.3 
timeshift = 0.01
nperseg = timewindow*fs
noverlap = (timewindow-timeshift)*fs
muscles = ['vl', 'rf', 'vm']


In [4]:
def get_time_per_rep(df_original: pd.DataFrame, df_frequencies:pd.DataFrame):
    df_frequencies['rep'] = None
    df_original['time'] = pd.to_datetime(df_original['time'])
    for t_set in df_original['rep'].unique():
        min_time = df_original[df_original['rep'] == t_set]['time'].min().timestamp()
        max_time = df_original[df_original['rep'] == t_set]['time'].max().timestamp()
        idx_min = (df_frequencies['time']-min_time).abs().idxmin()
        idx_max = (df_frequencies['time']-max_time).abs().idxmin()
        
        df_frequencies.loc[idx_min:idx_max, 'rep'] = t_set
        
    return df_frequencies
          
        

In [5]:
for training in df['training_id'].unique():
    for pp in df['pp'].unique():
        filtered = df[(df['pp'] == pp) & (df['training_id'] == training)]
        for t_set in filtered['set'].unique():
            if t_set != 0:
                filtered = filtered[filtered['set']==t_set]
            else: 
                continue
            for muscle in muscles:
                print(training, pp, t_set)
                f, t, Zxx = stft(filtered[f'Filtered_{muscle}'], fs, nperseg=nperseg, noverlap=noverlap)
                power_spectrum = np.abs(Zxx) ** 2
                median_frequencies = np.zeros(len(t))
                power = np.zeros(len(t))
             
                for i, time_frame in enumerate(power_spectrum.T):  # Transpose power_spectrum to iterate over time frames
                    # cumulative distribution function (CDF)
                    cum_sum = np.cumsum(time_frame)
                    power[i] = cum_sum[-1]
                    cum_sum /= cum_sum[-1]
                    median_index = np.argmax(cum_sum >= 0.5)
                    median_frequency = f[median_index]
                    median_frequencies[i] = median_frequency
                    
                
                data = {'time': np.arange(0,(len(median_frequencies)*timeshift), timeshift), f'MF_{muscle}': median_frequencies, f'Power_{muscle}': power/np.max(power)*100}
                for key, value in data.items():
                    print(f"Length of values for '{key}': {len(value)}")
                
                
                df_data = pd.DataFrame(data)
                
                df_frequencies = get_time_per_rep(filtered, df_data)
                df_frequencies['set'] = t_set 
                df_frequencies['training'] = training
                df_frequencies['pp'] = pp
                
                #sns.lmplot(x='Time', y='Median Frequency', data=df, height=6, aspect=2)
                plt.figure()
                sns.lineplot(x='time', y='MF_vl', data=df_frequencies, hue='rep')
                plt.show()
                
                plt.figure()
                sns.lineplot(data=filtered ,x='time', y='vl', hue='rep')
                #sns.lmplot(x='time', y='value', hue='variable',
                #           data=pd.melt(df_test, ['time']), height=6, aspect=2)
                plt.show()
                break
            break  
        break
    break


df_frequencies


            

LE_HYP_5x12 1 1
Length of values for 'time': 3131
Length of values for 'MF_vl': 3131
Length of values for 'Power_vl': 3131


,time,MF_vl,Power_vl,rep,set,training,pp
0,0.00,16.666667,0.002178,0.0,1,LE_HYP_5x12,1
1,0.01,16.666667,0.002490,0.0,1,LE_HYP_5x12,1
2,0.02,16.666667,0.002738,0.0,1,LE_HYP_5x12,1
3,0.03,16.666667,0.002902,0.0,1,LE_HYP_5x12,1
4,0.04,16.666667,0.002974,0.0,1,LE_HYP_5x12,1
...,...,...,...,...,...,...,...
3126,31.26,36.666667,0.079292,0.0,1,LE_HYP_5x12,1
3127,31.27,33.333333,0.083611,0.0,1,LE_HYP_5x12,1
3128,31.28,33.333333,0.085291,0.0,1,LE_HYP_5x12,1
3129,31.29,30.000000,0.083680,0.0,1,LE_HYP_5x12,1


**fourier transfomation**

In [6]:
def calculate_mpf(EMG, sampling_rate, N):
    FT = np.fft.fft(EMG, axis=0)
    psd = FT * np.conj(FT) # square of the FT --> power ((a+bi)(a−bi)=a²−(bi)²=a²−(−1)(b²)=a² +b²
    NFFT = len(FT)
    f = (np.arange(0, NFFT / 2) * sampling_rate) / N
    D_1 = 0
    N_1 = 0
    MPF = []
    weighted_sum = 0 
    total_power = 0 
    for j in np.arange(1, NFFT // 2):
        D_1 = D_1 + f[j] * psd[j]
        N_1 = N_1 + psd[j]
        MPF.append(D_1 / N_1)

        weighted_sum += f[j] * psd[j]  # Update the weighted sum
        total_power += psd[j]  # Update the total power

    median_power_freq = np.median(MPF)
    mean_frequency = weighted_sum / total_power 

    return f, psd[:len(f)], median_power_freq, mean_frequency

In [7]:
df['set'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [8]:
df['training_id'].unique()

array(['LE_HYP_5x12', 'LE_HYP_8x12', 'LE_MAX_5x4', 'LP_HYP_5x12'],
      dtype=object)

In [9]:
df_plot = df[(df['training_id'] == 'LE_HYP_5x12') & (df['pp']==3)]
print(len(df_plot))
f, psd, mf, meanf = calculate_mpf(df_plot[(df_plot['set']==1) & (df_plot['rep']==1)]['Filtered_vl'], 10, len(df_plot[(df_plot['set']==1) & (df_plot['rep']==1)]['Filtered_vl']))
print(np.real(mf))
psd = np.real(psd)

plt.figure(figsize=(10, 6))
plt.plot(f, psd[:len(f)], label='Median Power Frequency (MPF)')
plt.axvline(x=mf, color='r', linestyle='--', label='Median Power Frequency')
#plt.axhline(y=mean_frequency, color='g', linestyle='--', label='Mean Frequency')
plt.xlabel('Frequency (Hz)')
plt.ylabel('MPF')
plt.title('Median Power Frequency (MPF) and Mean Frequency')
plt.legend()
plt.grid(True)
plt.show()


257742
0.6364100013581409


c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\env\lib\site-packages\matplotlib\cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [10]:
data = {
            'mean' : [],
            'Median': [],
            'muscle': [],
            'rep':[],
            'set':[],
            'training':[],
            'pp':[],
        }

for training in df['training_id'].unique():
    for pp in df['pp'].unique():
        filtered = df[(df['pp'] == pp) & (df['training_id'] == training)]
        for t_set in filtered['set'].unique():
            if t_set != 0:
                #print(t_set, 'test')
                filtered_set = filtered[filtered['set']==t_set]
                for muscle in muscles:
                    for rep in filtered_set['rep'].unique():
                        if rep == 0:
                            continue
                        
                        filtered_rep = filtered_set[filtered_set['rep'] == rep]
                        if not np.isnan(filtered_rep[f'Filtered_{muscle}']).all():
                            f, psd, median_frequency, mean_frequency = calculate_mpf(filtered_rep[f'Filtered_{muscle}'], fs, len(filtered_rep[f'Filtered_{muscle}']))
                
                            data['mean'].append(mean_frequency.real)
                            data['Median'].append(median_frequency.real)
                            data['muscle'].append(muscle)
                            data['rep'].append(rep)
                            data['set'].append(t_set)
                            data['training'].append(training)
                            data['pp'].append(pp)
                            
                        else:
                            data['mean'].append(None)
                            data['Median'].append(None)
                            data['muscle'].append(muscle)
                            data['rep'].append(rep)
                            data['set'].append(t_set)
                            data['training'].append(training)
                            data['pp'].append(pp)
                                        
df_data = pd.DataFrame(data)
df_data.head(10)
                 
#df_data.to_csv('result_emg.csv')
                
               
                


,mean,Median,muscle,rep,set,training,pp
0,83.413294,81.901011,vl,1.0,1,LE_HYP_5x12,1
1,91.319888,89.981222,vl,2.0,1,LE_HYP_5x12,1
2,88.433817,87.000048,vl,3.0,1,LE_HYP_5x12,1
3,91.837537,90.447786,vl,4.0,1,LE_HYP_5x12,1
4,84.913564,83.976428,vl,5.0,1,LE_HYP_5x12,1
5,90.610566,89.568290,vl,6.0,1,LE_HYP_5x12,1
6,86.797078,85.774676,vl,7.0,1,LE_HYP_5x12,1
7,83.266540,82.210138,vl,8.0,1,LE_HYP_5x12,1
8,83.196951,82.285009,vl,9.0,1,LE_HYP_5x12,1
9,91.466612,90.321981,vl,10.0,1,LE_HYP_5x12,1


In [11]:
   #Garbage for plotting in the prevoiues bloxk of code
   
   
#  df_data = df_data[df_data['frequencys']>0]
                    #df_data['yf'] = abs(df_data['yf'])
#sns.lmplot(x='Time', y='Median Frequency', data=df, height=6, aspect=2)

                #
                #plt.axvline(median_frequency, linestyle='--', c='red')
                #plt.axvline(mean_frequency, linestyle='--', c='black')
               
                df_data = df_data[(df_data['rep'].str[0] == 1) | (df_data['rep'].str[0] == 12)]
                df['rep'] = df['rep'].astype(str)
                
                sns.lineplot(x='Frequency', y='Magnitude', data=df_data)#, hue=df['rep'].str[0])
                
                
              
                ##plt.axvline(mean, linestyle='--', c='black')

IndentationError: unexpected indent (1359776263.py, line 12)

In [ ]:
df_data.head(5)

,mean,Median,muscle,rep,set,training,pp
0,83.413294,81.901011,vl,1.0,1,LE_HYP_5x12,1
1,91.319888,89.981222,vl,2.0,1,LE_HYP_5x12,1
2,88.433817,87.000048,vl,3.0,1,LE_HYP_5x12,1
3,91.837537,90.447786,vl,4.0,1,LE_HYP_5x12,1
4,84.913564,83.976428,vl,5.0,1,LE_HYP_5x12,1


In [12]:
container = AnalysisContainer()
predictors = ['mean', 'Median']
#diff_types_training = df_data['training'].unique()
diff_types_training = ['LE_HYP_8x12']

for muscle in muscles:
    df_analysis = df_data[df_data['muscle'] == muscle] 
    for predictor in predictors: 
        fig, axs = plt.subplots(4, 8, figsize=(15, 15), sharey=True)
        plt.subplots_adjust(wspace=0.5, hspace=0.5)
        for i, training in enumerate(diff_types_training):
            results = AnalysingResults(f'mean_{predictor}_{training}_{muscle}')
            
            training_filter = (df_analysis['training'] == training)
            t_sets = df_analysis[training_filter]['set'].unique()
            filtered_df = df_analysis[training_filter]
            filtered_df = filtered_df[filtered_df['rep']>0]
            
            for j, t_set in enumerate(t_sets):
                df_to_process = filtered_df[filtered_df['set'] == t_set]
                ax = axs[i, j]
                
                for pp in set(df_data['pp']):
                    pp=3
                    df_pp_avg = df_to_process[df_to_process['pp']==pp]
                    results.update(pp, training, t_set, f'{muscle}_{predictor}', df_pp_avg, predictor, plot=True, ax=ax, title=f'set_{t_set}', y_label='MF')
                
            print(results)
            container.update(results)
            
        plt.suptitle(f'{muscle}_{predictor}')
        plt.legend()       
        plt.show()
                

output1 = container.df_with_diffrent_columns
output2 = container.df_with_type_column

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

In [15]:
avg_tabel = output2.groupby(['training','set', 'type_analysis']).agg(['mean', 'std']).reset_index()
output2

,pp,training,set,B0,B0_std,B1,B1_std,R2,type_analysis
0,1,LE_HYP_5x12,1,89.857223,2.656109,-0.515403,0.383014,0.167497,vl_mean
1,2,LE_HYP_5x12,1,85.936254,1.365671,-1.134044,0.185558,0.788810,vl_mean
2,3,LE_HYP_5x12,1,100.620047,1.752317,-1.682376,0.238093,0.833136,vl_mean
3,4,LE_HYP_5x12,1,86.232710,2.163715,-1.153216,0.293991,0.606097,vl_mean
4,5,LE_HYP_5x12,1,NaN,NaN,NaN,NaN,NaN,vl_mean
...,...,...,...,...,...,...,...,...,...
20,1,LP_HYP_5x12,5,131.537904,1.734858,-0.897637,0.235721,0.591858,vm_Median
21,2,LP_HYP_5x12,5,84.231470,2.066734,-0.206020,0.280814,0.051076,vm_Median
22,3,LP_HYP_5x12,5,111.511852,1.793080,-0.650553,0.243632,0.416234,vm_Median
23,4,LP_HYP_5x12,5,111.356873,1.776198,-0.614864,0.241338,0.393606,vm_Median


In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
model = smf.mixedlm("B0 ~ training", output2, groups=output2["pp"])
result = model.fit()

# Print the summary of the model
print(result.summary())

IndexError: index 615 is out of bounds for axis 0 with size 612

In [75]:
fig, ax = plt.subplots()
for training in output2['training'].unique():
    df_training = output2[output2['training'] == training]
    for pp in df_training['pp'].unique():
        df_pp = df_training[df_training['pp']==pp]
        df_pp = df_pp.reset_index(drop=True)
        sns.lineplot(data = df_pp, x='set', y='B1', ax=ax, c='grey', linestyle='--', errorbar=None)
        
    mean_all = df_training.groupby(['training','set', 'type_analysis']).mean().reset_index()
    mean_all = mean_all.reset_index(drop=True)
    sns.lineplot(data = mean_all, x='set', y='B1', ax=ax, c='red', errorbar=None)
    plt.show()
    break


In [77]:
for analysis in avg_tabel['type_analysis'].unique():
    for muscle in ['vl', 'rf', 'vm']:
        df_to_use = avg_tabel[avg_tabel['type_analysis'] == analysis]
        parameter_to_use = f'B1'
        #parameter_std = f'B1_std_{muscle}'

        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        axes = axes.flatten()
        y_min = df_to_use[parameter_to_use]['mean'].min() - 0.5 * df_to_use[parameter_to_use]['std'].max()
        y_max = df_to_use[parameter_to_use]['mean'].max() + 0.5 * df_to_use[parameter_to_use]['std'].max()

        for i, (type, group_data) in enumerate(df_to_use.groupby('training')):
            sns.lineplot(data=group_data, x='set', y=group_data[parameter_to_use]['mean'], label='mean', ax=axes[i])
            
            lower_bound = group_data[parameter_to_use]['mean'] - group_data[parameter_to_use]['std'] / 2
            upper_bound = group_data[parameter_to_use]['mean'] + group_data[parameter_to_use]['std'] / 2
            
            axes[i].fill_between(group_data['set'], lower_bound, upper_bound, color='blue', alpha=0.3, label='std')
            axes[i].text(0.5, -0.2, f"mean_R²: {round(group_data[f'R2']['mean'].mean(),4)}", transform=axes[i].transAxes, fontsize=10, ha='center')
            
            axes[i].set_ylabel(analysis.split('_')[1] + ' Frequency')
            axes[i].set_title(type)
            axes[i].legend()
            axes[i].set_ylim(y_min, y_max)

        fig.suptitle(f'{analysis} of {parameter_to_use}')
        #plt.show()
        plt.tight_layout()
        plt.savefig(f'{analysis}_{parameter_to_use}')

In [47]:
avg_tabel.head(5)

training set type_analysis   pp                    B0             \
                                 mean       std        mean        std   
0  LE_HYP_5x12   1     rf_Median  3.0  1.581139  109.235551  10.259875   
1  LE_HYP_5x12   1       rf_mean  3.0  1.581139  111.497537  10.698053   
2  LE_HYP_5x12   1     vl_Median  3.0  1.581139   89.597002   6.436622   
3  LE_HYP_5x12   1       vl_mean  3.0  1.581139   90.661559   6.874146   
4  LE_HYP_5x12   1     vm_Median  3.0  1.581139  104.318856  29.555928   

     B0_std                  B1              B1_std                  R2  \
       mean       std      mean       std      mean       std      mean   
0  2.277483  0.964802 -2.087440  0.612825  0.314294  0.131781  0.809441   
1  2.316320  0.973533 -2.228725  0.642844  0.319709  0.133175  0.824117   
2  1.965978  0.560529 -1.064442  0.454194  0.272619  0.085220  0.584402   
3  1.984453  0.553785 -1.121260  0.477179  0.275164  0.084438  0.598885   
4  2.978018  0.543406 -1.108458  0.149325  0.412518  0.089517  0.431953   

             
        std  
0  0.105599  
1  0.098967  
2  0.311394  
3  0.303915  
4  0.094654

In [49]:
avg_tabel.columns

MultiIndex([(     'training',     ''),
            (          'set',     ''),
            ('type_analysis',     ''),
            (           'pp', 'mean'),
            (           'pp',  'std'),
            (           'B0', 'mean'),
            (           'B0',  'std'),
            (       'B0_std', 'mean'),
            (       'B0_std',  'std'),
            (           'B1', 'mean'),
            (           'B1',  'std'),
            (       'B1_std', 'mean'),
            (       'B1_std',  'std'),
            (           'R2', 'mean'),
            (           'R2',  'std')],
           )

In [62]:

df_to_use = avg_tabel[avg_tabel['type_analysis'] == 'rf_Median']
df_corr = df_to_use[df_to_use['training'] == 'LE_HYP_5x12']

print('corr: ', df_corr['set'].corr(df_corr['B0']['mean'], method='spearman'))

corr:  -0.7999999999999999
